<a href="https://colab.research.google.com/github/dashsumit/Automated_Text_Summerization-/blob/main/Automated__Hybrid(Extractive_%2B_Abstractive_)_English_Text_Summerizatio_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extractive Text Summerization

In [1]:
# # for extractive summerization  we will use sentence importance to generate the extractive summary of the input text
# 1. we will tokenize the text and identify the ocuurence of important words
#    then we will calculate the importance of each word .

# 2. next we will assign weights two sentences  based on on the sum of the important  score of there words .

# 3. finally we will rank the sentences and select top and sentences from the text body  to form the summary

In [25]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
import pandas as pd
from heapq import nlargest,nsmallest

In [26]:
text = '''
At the heart of cutting-edge innovation is a silent revolution—artificial intelligence. It is not just a scientific field, but a dream come true: one in which machines can mimic, and even surpass, human intelligence. The main goal of artificial intelligence is to teach machines—to think, learn, and adapt—just as we do.

But how is this possible? The answer lies in the very heart of artificial intelligence, machine learning. Machine learning is a method by which machines learn from data. Just as a child learns to recognize faces, say words, or solve puzzles through experience, a machine learns from examples, patterns, and results.

Artificial intelligence is not limited to learning; it also extends to perceiving, reasoning, and acting. Machines can understand human language through natural language processing (NLP). They can analyze images and scenes through computer vision. They use neural networks to mimic the structure of the human brain—processing complex data through layers of “neurons.”

One of the most powerful architectures in machine learning is the deep learning model, where machines learn abstract features through layers of learning. It’s the technology that’s used to create voice assistants, smart translators, and even works of art.

But beyond algorithms or models, AI is a reflection of us—our creativity, our pursuit of reason, and our intelligence. With power comes responsibility, and ethical AI reminds us that we must build a future that is fair, transparent, and beneficial for all.

Machine learning and AI are not just technologies—they are a bridge between the human mind and the world of machines, with the potential to transform every aspect of life—from medicine to music, from education to exploration.
'''

In [27]:
len(text)

1750

Small Size englis model from spacy

In [28]:
nlp=spacy.load('en_core_web_sm')
nlp

applying the loaded pretrain model

In [29]:
doc = nlp(text)
doc


At the heart of cutting-edge innovation is a silent revolution—artificial intelligence. It is not just a scientific field, but a dream come true: one in which machines can mimic, and even surpass, human intelligence. The main goal of artificial intelligence is to teach machines—to think, learn, and adapt—just as we do.

But how is this possible? The answer lies in the very heart of artificial intelligence, machine learning. Machine learning is a method by which machines learn from data. Just as a child learns to recognize faces, say words, or solve puzzles through experience, a machine learns from examples, patterns, and results.

Artificial intelligence is not limited to learning; it also extends to perceiving, reasoning, and acting. Machines can understand human language through natural language processing (NLP). They can analyze images and scenes through computer vision. They use neural networks to mimic the structure of the human brain—processing complex data through layers of “ne

Code For tokenizations and removing stopwords from our text



In [30]:
# using list comprehension

tokens= [token.text.lower() for token in doc
         if not token.is_stop and
         not token.is_punct and
         token.text != '\n']

In [31]:
tokens

['heart',
 'cutting',
 'edge',
 'innovation',
 'silent',
 'revolution',
 'artificial',
 'intelligence',
 'scientific',
 'field',
 'dream',
 'come',
 'true',
 'machines',
 'mimic',
 'surpass',
 'human',
 'intelligence',
 'main',
 'goal',
 'artificial',
 'intelligence',
 'teach',
 'machines',
 'think',
 'learn',
 'adapt',
 '\n\n',
 'possible',
 'answer',
 'lies',
 'heart',
 'artificial',
 'intelligence',
 'machine',
 'learning',
 'machine',
 'learning',
 'method',
 'machines',
 'learn',
 'data',
 'child',
 'learns',
 'recognize',
 'faces',
 'words',
 'solve',
 'puzzles',
 'experience',
 'machine',
 'learns',
 'examples',
 'patterns',
 'results',
 '\n\n',
 'artificial',
 'intelligence',
 'limited',
 'learning',
 'extends',
 'perceiving',
 'reasoning',
 'acting',
 'machines',
 'understand',
 'human',
 'language',
 'natural',
 'language',
 'processing',
 'nlp',
 'analyze',
 'images',
 'scenes',
 'computer',
 'vision',
 'use',
 'neural',
 'networks',
 'mimic',
 'structure',
 'human',
 'brain

Alternative approch tokenizations and removing stopwords from our text using pos taging

In [32]:
tokens1=[]
stopwords = list(STOP_WORDS)
allowed_pos = ['ADJ','PROPN','VERB','NOUN']
for token in doc :
  if token.text in stopwords or token.text in punctuation:
    continue
  if token.pos_ in allowed_pos:
    tokens1.append(token.text)

In [33]:
tokens1

['heart',
 'cutting',
 'edge',
 'innovation',
 'silent',
 'revolution',
 'artificial',
 'intelligence',
 'scientific',
 'field',
 'dream',
 'come',
 'true',
 'machines',
 'mimic',
 'surpass',
 'human',
 'intelligence',
 'main',
 'goal',
 'artificial',
 'intelligence',
 'teach',
 'machines',
 'think',
 'learn',
 'adapt',
 'possible',
 'answer',
 'lies',
 'heart',
 'artificial',
 'intelligence',
 'machine',
 'learning',
 'Machine',
 'learning',
 'method',
 'machines',
 'learn',
 'data',
 'child',
 'learns',
 'recognize',
 'faces',
 'words',
 'solve',
 'puzzles',
 'experience',
 'machine',
 'learns',
 'examples',
 'patterns',
 'results',
 'Artificial',
 'intelligence',
 'limited',
 'learning',
 'extends',
 'perceiving',
 'reasoning',
 'acting',
 'Machines',
 'understand',
 'human',
 'language',
 'natural',
 'language',
 'processing',
 'NLP',
 'analyze',
 'images',
 'scenes',
 'computer',
 'vision',
 'use',
 'neural',
 'networks',
 'mimic',
 'structure',
 'human',
 'brain',
 'processing',


Frequency of the sentences

In [34]:
# for extractive we have to find the sentences that have most  important word in them by focusing on sentences with the keywords appearing most often .ImportWarning

# we create the summary that capture the main idea of the summary .

In [35]:
word_freq=Counter(tokens)
word_freq

Counter({'heart': 2,
         'cutting': 1,
         'edge': 1,
         'innovation': 1,
         'silent': 1,
         'revolution': 1,
         'artificial': 4,
         'intelligence': 6,
         'scientific': 1,
         'field': 1,
         'dream': 1,
         'come': 1,
         'true': 1,
         'machines': 6,
         'mimic': 2,
         'surpass': 1,
         'human': 4,
         'main': 1,
         'goal': 1,
         'teach': 1,
         'think': 1,
         'learn': 3,
         'adapt': 1,
         '\n\n': 5,
         'possible': 1,
         'answer': 1,
         'lies': 1,
         'machine': 5,
         'learning': 7,
         'method': 1,
         'data': 2,
         'child': 1,
         'learns': 2,
         'recognize': 1,
         'faces': 1,
         'words': 1,
         'solve': 1,
         'puzzles': 1,
         'experience': 1,
         'examples': 1,
         'patterns': 1,
         'results': 1,
         'limited': 1,
         'extends': 1,
         'perce

In [36]:
#pos approach
word_freq1=Counter(tokens1)
word_freq1

Counter({'heart': 2,
         'cutting': 1,
         'edge': 1,
         'innovation': 1,
         'silent': 1,
         'revolution': 1,
         'artificial': 3,
         'intelligence': 6,
         'scientific': 1,
         'field': 1,
         'dream': 1,
         'come': 1,
         'true': 1,
         'machines': 5,
         'mimic': 2,
         'surpass': 1,
         'human': 4,
         'main': 1,
         'goal': 1,
         'teach': 1,
         'think': 1,
         'learn': 3,
         'adapt': 1,
         'possible': 1,
         'answer': 1,
         'lies': 1,
         'machine': 3,
         'learning': 7,
         'Machine': 2,
         'method': 1,
         'data': 2,
         'child': 1,
         'learns': 2,
         'recognize': 1,
         'faces': 1,
         'words': 1,
         'solve': 1,
         'puzzles': 1,
         'experience': 1,
         'examples': 1,
         'patterns': 1,
         'results': 1,
         'Artificial': 1,
         'limited': 1,
         

In [37]:
max_freq= max(word_freq.values())
max_freq

7

In [38]:
#pos approach
max_freq1= max(word_freq1.values())
max_freq1

7

Normalizing the values

In [39]:
for word in word_freq.keys():
  word_freq[word] = word_freq[word] / max_freq

word_freq

Counter({'heart': 0.2857142857142857,
         'cutting': 0.14285714285714285,
         'edge': 0.14285714285714285,
         'innovation': 0.14285714285714285,
         'silent': 0.14285714285714285,
         'revolution': 0.14285714285714285,
         'artificial': 0.5714285714285714,
         'intelligence': 0.8571428571428571,
         'scientific': 0.14285714285714285,
         'field': 0.14285714285714285,
         'dream': 0.14285714285714285,
         'come': 0.14285714285714285,
         'true': 0.14285714285714285,
         'machines': 0.8571428571428571,
         'mimic': 0.2857142857142857,
         'surpass': 0.14285714285714285,
         'human': 0.5714285714285714,
         'main': 0.14285714285714285,
         'goal': 0.14285714285714285,
         'teach': 0.14285714285714285,
         'think': 0.14285714285714285,
         'learn': 0.42857142857142855,
         'adapt': 0.14285714285714285,
         '\n\n': 0.7142857142857143,
         'possible': 0.14285714285714285,


In [40]:
#pos approach

for word in word_freq1.keys():
  word_freq1[word] = word_freq1[word] / max_freq1

word_freq1

Counter({'heart': 0.2857142857142857,
         'cutting': 0.14285714285714285,
         'edge': 0.14285714285714285,
         'innovation': 0.14285714285714285,
         'silent': 0.14285714285714285,
         'revolution': 0.14285714285714285,
         'artificial': 0.42857142857142855,
         'intelligence': 0.8571428571428571,
         'scientific': 0.14285714285714285,
         'field': 0.14285714285714285,
         'dream': 0.14285714285714285,
         'come': 0.14285714285714285,
         'true': 0.14285714285714285,
         'machines': 0.7142857142857143,
         'mimic': 0.2857142857142857,
         'surpass': 0.14285714285714285,
         'human': 0.5714285714285714,
         'main': 0.14285714285714285,
         'goal': 0.14285714285714285,
         'teach': 0.14285714285714285,
         'think': 0.14285714285714285,
         'learn': 0.42857142857142855,
         'adapt': 0.14285714285714285,
         'possible': 0.14285714285714285,
         'answer': 0.142857142857142

Sentence Tokenizations

In [41]:
sent_token = [sent.text for sent in doc.sents]
sent_token

['\nAt the heart of cutting-edge innovation is a silent revolution—artificial intelligence.',
 'It is not just a scientific field, but a dream come true: one in which machines can mimic, and even surpass, human intelligence.',
 'The main goal of artificial intelligence is to teach machines—to think, learn, and adapt—just as we do.\n\n',
 'But how is this possible?',
 'The answer lies in the very heart of artificial intelligence, machine learning.',
 'Machine learning is a method by which machines learn from data.',
 'Just as a child learns to recognize faces, say words, or solve puzzles through experience, a machine learns from examples, patterns, and results.\n\n',
 'Artificial intelligence is not limited to learning; it also extends to perceiving, reasoning, and acting.',
 'Machines can understand human language through natural language processing (NLP).',
 'They can analyze images and scenes through computer vision.',
 'They use neural networks to mimic the structure of the human br

writing logic to calculate sentences score based on word frequencies

In [42]:
sent_score= {}
for sent in sent_token:
  for word in sent.split():
    if word.lower() in word_freq.keys():
      if sent not in sent_score.keys():
        sent_score[sent] = word_freq[word.lower()]
      else:
        sent_score[sent] += word_freq[word]
    print(word)


At
the
heart
of
cutting-edge
innovation
is
a
silent
revolution—artificial
intelligence.
It
is
not
just
a
scientific
field,
but
a
dream
come
true:
one
in
which
machines
can
mimic,
and
even
surpass,
human
intelligence.
The
main
goal
of
artificial
intelligence
is
to
teach
machines—to
think,
learn,
and
adapt—just
as
we
do.
But
how
is
this
possible?
The
answer
lies
in
the
very
heart
of
artificial
intelligence,
machine
learning.
Machine
learning
is
a
method
by
which
machines
learn
from
data.
Just
as
a
child
learns
to
recognize
faces,
say
words,
or
solve
puzzles
through
experience,
a
machine
learns
from
examples,
patterns,
and
results.
Artificial
intelligence
is
not
limited
to
learning;
it
also
extends
to
perceiving,
reasoning,
and
acting.
Machines
can
understand
human
language
through
natural
language
processing
(NLP).
They
can
analyze
images
and
scenes
through
computer
vision.
They
use
neural
networks
to
mimic
the
structure
of
the
human
brain—processing
complex
data
through
layers
of
“neuro

In [43]:
sent_score

{'\nAt the heart of cutting-edge innovation is a silent revolution—artificial intelligence.': 0.5714285714285714,
 'It is not just a scientific field, but a dream come true: one in which machines can mimic, and even surpass, human intelligence.': 1.857142857142857,
 'The main goal of artificial intelligence is to teach machines—to think, learn, and adapt—just as we do.\n\n': 1.857142857142857,
 'The answer lies in the very heart of artificial intelligence, machine learning.': 1.8571428571428572,
 'Machine learning is a method by which machines learn from data.': 3.142857142857143,
 'Just as a child learns to recognize faces, say words, or solve puzzles through experience, a machine learns from examples, patterns, and results.\n\n': 1.8571428571428568,
 'Artificial intelligence is not limited to learning; it also extends to perceiving, reasoning, and acting.': 1.714285714285714,
 'Machines can understand human language through natural language processing (NLP).': 2.571428571428571,
 'Th

Creating a dataframe in pandas

In [44]:
df =pd.DataFrame(sent_score.items(),columns=['sentence','score'])
df

,sentence,score
0,\nAt the heart of cutting-edge innovation is a...,0.571429
1,"It is not just a scientific field, but a dream...",1.857143
2,The main goal of artificial intelligence is to...,1.857143
3,The answer lies in the very heart of artificia...,1.857143
4,Machine learning is a method by which machines...,3.142857
5,"Just as a child learns to recognize faces, say...",1.857143
6,Artificial intelligence is not limited to lear...,1.714286
7,Machines can understand human language through...,2.571429
8,They can analyze images and scenes through com...,0.571429
9,They use neural networks to mimic the structur...,2.142857


In [45]:
#as we are doing extractive summarization , so we have to select top most scores

In [46]:
number_of_sentences = 3
n= nlargest(number_of_sentences,sent_score,key =sent_score.get)
# key =sent_score.get
# use to extract the score from df to comapre the largest
n

['One of the most powerful architectures in machine learning is the deep learning model, where machines learn abstract features through layers of learning.',
 'Machine learning and AI are not just technologies—they are a bridge between the human mind and the world of machines, with the potential to transform every aspect of life—from medicine to music, from education to exploration.\n',
 'Machine learning is a method by which machines learn from data.']

In [58]:
extractive_summerization = " " .join(n)
extractive_summerization

'One of the most powerful architectures in machine learning is the deep learning model, where machines learn abstract features through layers of learning. Machine learning and AI are not just technologies—they are a bridge between the human mind and the world of machines, with the potential to transform every aspect of life—from medicine to music, from education to exploration.\n Machine learning is a method by which machines learn from data.'

In [68]:
len(extractive_summerization)

444

# End Of Extractive Text Summerrization

# Abstractive text Summerization

In [48]:
from transformers import pipeline

In [49]:
summerizer = pipeline("summarization", model= 't5-base', tokenizer='t5-base', framework='pt')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cpu


Applying in to the model

In [50]:
summery = summerizer(text, max_length=50, min_length=30, do_sample=False)

In [51]:
summery

[{'summary_text': 'at the heart of cutting-edge innovation is a silent revolution—artificial intelligence . the main goal of artificial intelligence is to teach machines to think, learn, and adapt . machine learning extends to perceiving,'}]

printing summary into proper format

In [52]:
print(summery[0]['summary_text']) # Changed 'summery_text' to 'summary_text'
# summary_text = summery[0]['summary_text']
# for line in summary_text.split('\n'):
#     print(line)

# summary_text = " ".join(summary_text.split())
# summary_text

at the heart of cutting-edge innovation is a silent revolution—artificial intelligence . the main goal of artificial intelligence is to teach machines to think, learn, and adapt . machine learning extends to perceiving,


In [53]:
summary_text=summery[0]['summary_text']
summary_text

'at the heart of cutting-edge innovation is a silent revolution—artificial intelligence . the main goal of artificial intelligence is to teach machines to think, learn, and adapt . machine learning extends to perceiving,'

In [54]:
len(summary_text)

219

# End Of Abstractive Summerization

# Hybrid(Extractive + Abstractive) Summerization

In [60]:
# Hybrid summarization (simple concatenation)
hybrid_summary =  extractive_summerization + " " + summary_text
hybrid_summary

'One of the most powerful architectures in machine learning is the deep learning model, where machines learn abstract features through layers of learning. Machine learning and AI are not just technologies—they are a bridge between the human mind and the world of machines, with the potential to transform every aspect of life—from medicine to music, from education to exploration.\n Machine learning is a method by which machines learn from data. at the heart of cutting-edge innovation is a silent revolution—artificial intelligence . the main goal of artificial intelligence is to teach machines to think, learn, and adapt . machine learning extends to perceiving,'

In [61]:
len(hybrid_summary)

664